In [4]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI 

## 기본 세팅. llm, parser
load_dotenv()

llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
output_parser = StrOutputParser()

## 문제 1-1 기본 체인 만들기
사용자가 재료를 입력하면 그 재료로 만들 수 있는 요리를 추천해주는 간단한 AI 요리사를 만들어보세요.

### 요구사항

1. PromptTemplate을 사용하여 프롬프트 작성
2. 사용자가 입력한 재료를 받아서 요리 추천
3. ChatOpenAI 모델 사용
4. StrOutputParser로 결과를 문자열로 출력
5. LCEL(|) 문법을 사용하여 체인 연결



In [5]:
template = PromptTemplate.from_template("{ingredients} 이 재료들을 가지고 만들 수 있는 요리를 한가지 추천해줘")


chef_chain = template | llm | output_parser

ingredients_input = input("재료를 입력해주세요 > ")
chef_response = chef_chain.invoke(ingredients_input)
print(chef_response)




**감자와 양파를 사용한 간단한 요리: 감자 양파전**

### 감자 양파전

*   **재료:**

    *   큰 감자 2개
    *   큰 양파 1/2개
    *   소금 1/2 티스푼
    *   후추 1/4 티스푼
    *   밀가루 1 티스푼
    *   물 1 티스푼
    *   식용유 2 티스푼
*   **방법:**

    1.  감자와 양파를 깨끗이 씻고 껍질을 벗긴 다음, 감자는 강판에 갈고 양파는 잘게 썰어주세요.
    2.  감자와 양파를 큰 그릇에 넣고 소금, 후추로 간을 한 후 잘 섞어주세요.
    3.  밀가루와 물을 섞어 감자와 양파에 넣고 잘 섞어주세요.
    4.  프라이팬에 식용유를 두르고 혼합물을 한 국자씩 올려서 전처럼 만들어주세요.
    5.  각 면을 황금빛 갈색이 될 때까지 구워주세요.
    6.  완성된 감자 양파전을 따뜻한 소스에 찍어 먹거나 그대로 즐기세요.

이 요리는 간단하면서도 맛있고 영양가도 높습니다. 감자의 고소함과 양파의 달콤함이 조화롭게 어우러져 누구나 쉽게 만들 수 있는 요리입니다.


## 문제 1-2 : 2단계 체인 만들기 - 영화 추천 시스템 
사용자가 좋아하는 장르를 입력하면, 영화를 추천하고 그 영화의 정보(감독, 줄거리, 등장인물)들을  알려주는 2단계 체인을 구현해보세요.

### 요구사항
1. 1단계 체인: 장르를 입력받아 영화 1편 추천
2. 2단계 체인: 추천받은 영화의 3줄 줄거리 요약 제공
3. ChatPromptTemplate 사용 :“system”  과 “human” 메시지를 지정합니다.
4. 두 체인을 LCEL로 연결
5. 각 단계의 결과를 모두 출력하여 과정 확인



In [13]:
movie_template = ChatPromptTemplate.from_messages([
    ("system", "너는 영화감독이야."),
    ("human", "{genre} 장르의 추천할만한 영화를 제목만 하나 출력해줘")
])



summation_template= ChatPromptTemplate.from_messages([
     ("human", "{movie} 영화의 줄거리를 3줄 요약해줘")
])


movie_chain = movie_template | llm | output_parser
summation_template = summation_template | llm | output_parser
movie_summation_chain = movie_chain | summation_template

movie_genre_input = input("영화 장르를 입력해주세요 입력해주세요 > ")
movie_response = movie_summation_chain.invoke(movie_genre_input)
print(movie_response)

공포 영화 '샤이닝'은 가족과 떨어져 있는 동안 호텔의 악령에 의해 미치게 되는 아버지 잭과, 그의 아내 웬디, 그리고 딸 데이지의 이야기를 다룹니다. 잭은 호텔에서 일하게 되는데, 호텔에는 과거에 끔찍한 사건이 발생했던 곳입니다. 호텔의 악령에 의해 잭이 미치게 되자, 웬디와 데이지는 호텔에서 탈출하려고 하지만, 잭은 그들을 막고 공격하기 시작합니다.


## 문제 1-3 : FewShotPromptTemplate과 시스템 메시지 활용 
FewShotPromptTemplate을 사용하여 뉴스 기사에서 핵심 키워드 3개를 추출하는 시스템을 구현해보세요. 주어진 예시들을 참고하여 일관된 형식으로 키워드를 추출해야 합니다.
### 요구사항
1. FewShotPromptTemplate 사용
2. 최소 3개의 예시(examples) 포함
3. 뉴스 텍스트에서 핵심 키워드 3개 추출
4. 일관된 출력 형식 유지
5. 다양한 분야의 뉴스로 테스트


In [6]:
examples = [
    {
        "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.",
        "keywords": "세계보건기구, 건강위기, 국제협력"
    },
    {
        "news": "기후 변화로 인해 전 세계적으로 극심한 날씨 현상이 증가하고 있다. 폭염, 가뭄, 홍수 등이 빈번해지면서 식량 안보와 생태계에 심각한 위협을 가하고 있으며, 각국 정부는 이에 대한 비상 계획을 수립 중이다.",
        "keywords": "기후 변화, 날씨 현상, 식량 안보"
    },
    {
        "news": "국내 자동차 제조사들이 전기차 판매량 증대를 위해 다양한 전략을 모색하고 있다. 보조금 확대, 충전 인프라 확충, 배터리 기술 개발 등 정부와 기업의 협력이 중요하다고 전문가들은 입을 모았다.",
        "keywords": "자동차 제조사, 전기차, 보조금"
    },
    {
        "news": "새로운 디지털 교육 플랫폼이 학교 현장에서 큰 호응을 얻고 있다. 개인 맞춤형 학습 콘텐츠와 AI 기반의 평가 시스템을 통해 학생들의 학업 성취도를 높이는 데 기여하고 있다는 평가다.",
        "keywords": "디지털 교육, 학습 플랫폼, AI"
    }
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "뉴스 키워드 추출 전문가입니다. 핵심 키워드 3개를 추출하세요."),
    few_shot_prompt, 
    ("human", "{news_input}"), 
    ("ai", "키워드:") 
])

news_chain = final_prompt | llm | output_parser
news_input = input("뉴스 > ")
news_response = news_chain.invoke(news_input)
print(news_response)



 제미나이, AI, 개발자
